In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from test_tool import multiply, load_cv_info
from CV import tailor_CV

# Create the agent
memory = MemorySaver()
model = init_chat_model("openai:gpt-4")
tools = [multiply, load_cv_info, tailor_CV]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Generate a tailored CV.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Generate a tailored CV.
================================== Ai Message ==================================
Tool Calls:
  tailor_CV (call_X2t9fuyYDg6cXKOlfDD4zJ4A)
 Call ID: call_X2t9fuyYDg6cXKOlfDD4zJ4A
  Args:
================================= Tool Message =================================
Name: tailor_CV

Fergal Riordan  
Ireland (Willing to Relocate) | +353 838754381 | fergalriordan333@gmail.com | Portfolio Website | GitHub | LinkedIn  

PROFILE  
Software Engineer with a First-Class Honours MEng in Electronic & Computer Engineering from Trinity College Dublin. Experience spans Agentic AI development for a Partner Relationship Management (PRM) platform, RLHF data annotation for LLMs, and generative AI research in CycleGAN architectures. Proficient in Python and PyTorch, with a passion for building responsible, efficient AI solutions.  

TECHNICAL SKILLS  
Key Competencies: Agentic AI, Generative AI, Prom

In [1]:
# Example of calling the CV tailoring prompt

from openai import OpenAI
client = OpenAI()

response = client.responses.create(
  prompt={
    "id": "pmpt_68b0c358291c81968a00e9414e386276009a458f13092457",
    "version": "2"
  }
)